In [5]:
from mlflow.tracking import MlflowClient

In [6]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [7]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [8]:
client.search_experiments()

[<Experiment: artifact_location='/Users/Jake/Olvin/dev/mlops-zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1682610782427, experiment_id='2', last_update_time=1682610782427, lifecycle_stage='active', name='my-test-experiment', tags={}>,
 <Experiment: artifact_location='/Users/Jake/Olvin/dev/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1682585772380, experiment_id='1', last_update_time=1682585772380, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [9]:
client.create_experiment("my-test-experiment")

MlflowException: Experiment(name=my-test-experiment) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) UNIQUE constraint failed: experiments.name
[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (?, ?, ?, ?, ?)]
[parameters: ('my-test-experiment', None, 'active', 1682614351442, 1682614351442)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [13]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids="1",
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [14]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse = {run.data.metrics['rmse']:.4f}")

run id: 8cd4bd90c153414996e8f26aea4a875d, rmse = 5.7852
run id: 6c29efb995de456691a402a9c2b3794b, rmse = 5.7852
run id: 98ec9262ecee4df39e7f14b8d2d874c1, rmse = 5.7852
run id: 9cdfff70fdf0430e8c546dcee236f1ea, rmse = 5.7946
run id: 32564d13d26446a7b0b01320c24ff194, rmse = 5.8020


In [15]:
model_name = "nyc-taxi-regressor"

In [16]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [17]:
run_id = runs[0].info.run_id
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name=model_name)

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/04/27 17:52:50 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 4
Created version '4' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1682614370245, current_stage='None', description=None, last_updated_timestamp=1682614370245, name='nyc-taxi-regressor', run_id='8cd4bd90c153414996e8f26aea4a875d', run_link=None, source='/Users/Jake/Olvin/dev/mlops-zoomcamp/02-experiment-tracking/mlruns/1/8cd4bd90c153414996e8f26aea4a875d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [18]:
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: Staging
version: 4, stage: None


In [19]:
model_version = 3
model_stage = "Staging"

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=model_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1682611733848, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2023-04-27', last_updated_timestamp=1682614372461, name='nyc-taxi-regressor', run_id='8cd4bd90c153414996e8f26aea4a875d', run_link=None, source='/Users/Jake/Olvin/dev/mlops-zoomcamp/02-experiment-tracking/mlruns/1/8cd4bd90c153414996e8f26aea4a875d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [20]:
from datetime import datetime

In [21]:
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {model_stage} on {datetime.today().date()}"
)

<ModelVersion: aliases=[], creation_timestamp=1682611733848, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2023-04-27', last_updated_timestamp=1682614374082, name='nyc-taxi-regressor', run_id='8cd4bd90c153414996e8f26aea4a875d', run_link=None, source='/Users/Jake/Olvin/dev/mlops-zoomcamp/02-experiment-tracking/mlruns/1/8cd4bd90c153414996e8f26aea4a875d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [31]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df['duration'] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime).dt.total_seconds() / 60
    df = df[((df.duration >= 1) & (df.duration <= 60))]
    categorical = ['PULocationID', 'DOLocationID']

    df[categorical] = df[categorical].astype(str)

    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + "_" + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient="records")
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [23]:
df = read_dataframe("data/green_tripdata_2022-03.parquet")

In [24]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

/var/folders/59/c0z2hyvn0bq2jktc5gbfxtdw0000gq/T/ipykernel_90255/2852634549.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'/Users/Jake/Olvin/dev/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [25]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [26]:
X_test = preprocess(df, dv)

In [27]:
target = "duration"
y_test = df[target].values

In [32]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

[18:00:50] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 4.99 s, sys: 61.3 ms, total: 5.06 s
Wall time: 932 ms


{'rmse': 5.848923684655217}